May: **Research Questions 2.1** (Ideal and Real)

# Which of the three upwelling mechanisms (time dependence, advection, convergence of isobaths, coastal trapped wave) provides the strongest contribution to upwelling in Mackenzie Canyon?

Where does upwelling occur?

In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/upwelling_depth/')
import upwelling_functions

import warnings
warnings.filterwarnings("ignore")

# Functions

In [2]:
def plot_original_depth_hourly(depth_upwelled, tmask, deptht, dep_ind_slice, actual_hours, day, levels, kind, case):
    cmap, norm = cmo.cm.delta, upwelling_functions.MidpointNormalize(midpoint=0)
    cmap.set_bad('silver')

    vmin, vmax = int(np.min(depth_upwelled)), int(np.max(depth_upwelled))
    xs = np.arange(tmask.shape[-1])
    ys = np.arange(tmask.shape[-2])

    fig, axes = plt.subplots(4, 3, figsize = (20, 28)) #
    for ax, n in zip(axes.flatten(), np.arange(12)): #
        p = ax.pcolormesh(xs, ys, depth_upwelled[n, :, :], cmap=cmap, vmin=vmin, vmax=vmax, norm=norm)
        cs = ax.contour(xs, ys, depth_upwelled[n, :, :], levels = levels, colors='k', alpha=0.8)
        ax.clabel(cs, inline=1, fontsize=10, fmt ='%r', colors='k')
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect(aspect='equal')
        ax.set_xlim([0, xs[-1]])
        ax.set_ylim([0, ys[-1]])
        ax.set_title('Hour ' + str(actual_hours[n]), fontsize=20) #
        cbar = fig.colorbar(p, ax=ax, fraction=0.05, orientation='horizontal', pad=0.009)
        cbar.ax.tick_params(labelsize=13)
        tick_locator = ticker.MaxNLocator(nbins=15)
        cbar.locator = tick_locator
        cbar.update_ticks()

    fig.tight_layout(w_pad=1.2, h_pad=0.01)
    fig.suptitle('Day ' + str(day) + ' - ' + kind + ' - ' + case +': ' + 'displacement depth [m] at depth = '+
                 str(int(deptht[dep_ind_slice]))+' m', fontsize=25)
    plt.subplots_adjust(top=0.96)

    return fig

In [3]:
def execute_functions(dirname, fname, meshmaskname, vosaline_ref_profile, dep_ind_slices, time_s, time_f, levels):
    
    hour_inds = np.arange(1, 24)[::2]
    inds = np.arange(len(hour_inds))
    actual_hours = hour_inds + time_s
    day = int(round(actual_hours[-1]/24))
    
    print('hour_inds', hour_inds)
    print('inds', inds)
    print('actual_hours', actual_hours)
    print('day', day)

    for dep_ind_slice in dep_ind_slices:
        vosaline0, sozotaux, deptht, tmask = upwelling_functions.get_vars_salt(dirname, fname, meshmaskname, dep_ind_slice, time_s, time_f, None)

        vosaline = np.full([len(hour_inds), vosaline0.shape[-2], vosaline0.shape[-1]], np.nan)
        
        for i, hour_ind in zip(inds, hour_inds):
            vosaline[i, :, :] = vosaline0[hour_ind, :, :]

        depth_ind_hourly = upwelling_functions.get_daily_depth_ind(vosaline, vosaline_ref_profile)
        depth_m_hourly, depth_upwelled = upwelling_functions.get_daily_depth_m(deptht, dep_ind_slice, depth_ind_hourly, tmask)

        fig = plot_original_depth_hourly(depth_upwelled, tmask, deptht, dep_ind_slice, actual_hours, day, levels, kind, case)
        fig.savefig('../writing_images/displaced_dep_'+kind+'_'+case+'_'+str(dep_ind_slice)+'_day'+str(day)+'.png', dpi=100, bbox_inches='tight')
        plt.close(fig) 
        
    return

# Constants

In [4]:
vosaline_ref = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/salinity/salinity_for_agrif.nc')['vosaline'][:]
vosaline_ref_profile = vosaline_ref[0, :, 0, 0]

fname = '1_MCKNZ_1h_20170101_201701*'
meshmaskname = '1_mesh_mask.nc'

levels = [10, 20, 40, 60, 80, 100, 120]
dep_ind_slices = [1, 9, 16]

# Day 1

In [5]:
time_s = 0
time_f = time_s+24

In [6]:
kind = 'ideal'
case = 'base'
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

execute_functions(dirname, fname, meshmaskname, vosaline_ref_profile, dep_ind_slices, time_s, time_f, levels)

hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]
day 1
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


In [7]:
kind = 'real'
case = 'base'
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

execute_functions(dirname, fname, meshmaskname, vosaline_ref_profile, dep_ind_slices, time_s, time_f, levels)

hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [ 1  3  5  7  9 11 13 15 17 19 21 23]
day 1
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


# Day 2

In [8]:
time_s = 24
time_f = time_s+24

In [9]:
kind = 'ideal'
case = 'base'
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

execute_functions(dirname, fname, meshmaskname, vosaline_ref_profile, dep_ind_slices, time_s, time_f, levels)

hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]
day 2
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


In [10]:
kind = 'real'
case = 'base'
dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

execute_functions(dirname, fname, meshmaskname, vosaline_ref_profile, dep_ind_slices, time_s, time_f, levels)

hour_inds [ 1  3  5  7  9 11 13 15 17 19 21 23]
inds [ 0  1  2  3  4  5  6  7  8  9 10 11]
actual_hours [25 27 29 31 33 35 37 39 41 43 45 47]
day 2
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
